In [1]:
import pickle
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer, CrossEncoder
import os
from pathlib import Path
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import DirectoryLoader, TextLoader

c:\Users\Admin\miniconda3\envs\Rag\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
try:
    from langchain_cohere import CohereRerank
    COHERE_AVAILABLE = True
except ImportError:    
    COHERE_AVAILABLE = False

In [2]:
# CONFIG = {
#     "data_path": "dulieu_daotao",
#     "vectorstore_path": "faiss_index",
#     "embedding_model": "sentence-transformers/all-MiniLM-L6-v2",
#     "retriever_k": 3
# }

In [3]:
# # Giả sử code tạm phần data của a Quân

# def prepare_vector_database(config):
#     vectorstore_path = Path(config["vectorstore_path"])
#     data_path = Path(config["data_path"])

#     if (vectorstore_path.exists()):
#         print("csdl vector đã tồn tại. Bỏ qua tạo mới")
#         return
    
#     if not data_path.exists():
#         data_path.mkdir(parents=True, exist_ok=True)
#         (data_path / "hoc_phi.txt").write_text("Quy định về học phí năm học 2024-2025 cho sinh viên ngành Công nghệ thông tin là 30 triệu đồng/năm. Sinh viên có thể đóng theo hai kỳ. Ngành Quản trị kinh doanh có mức học phí là 28 triệu đồng/năm.", encoding='utf-8')
#         (data_path / "lich_nghi.txt").write_text("Trường thông báo lịch nghỉ lễ Quốc Khánh. Toàn thể sinh viên được nghỉ vào ngày 2 tháng 9. Lịch thi cuối kỳ sẽ được thông báo sau ngày nghỉ lễ.", encoding='utf-8')
#         (data_path / "thu_tuc.txt").write_text("Thủ tục nhập học cho tân sinh viên yêu cầu nộp bản sao công chứng học bạ THPT và giấy báo trúng tuyển trước ngày 15 tháng 8. Thủ tục đăng ký thi lại cần hoàn thành đơn trực tuyến trên cổng thông tin.", encoding='utf-8')

#     # Tải dữ liệu
#     loader = DirectoryLoader(
#         str(data_path),
#         glob="**/*.txt",
#         loader_cls=TextLoader,
#         loader_kwargs={'encoding': 'utf-8'}
#     )
#     documents = loader.load()

#     #Chia chunk
#     text_splitter = RecursiveCharacterTextSplitter(
#         chunk_size = 500,
#         chunk_overlap = 50
#     )
#     docs = text_splitter.split_documents(documents)
#     print("Đã chia {len(documents)} tài liệu thành {len(docs)} chunks")

#     embeddings_model = HuggingFaceEmbeddings(
#         model_name = config["embedding_model"],
#         model_kwargs = {'device' : 'cpu'}
#     )

#     vectorstore = FAISS.from_documents(docs, embeddings_model)
#     vectorstore.save_local(config["vectorstore_path"])


In [4]:
# # Tích hợp reranking 
# from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
# from langchain_cohere import CohereRerank

# def create_reranking_retriever(query_system: QuerySystem):
#     try:
#         cohere_api_key = os.environ.get("COHERE_API_KEY")
#         if not cohere_api_key:
#             raise ValueError("Không thấy cohere api")
        
#         compressor = CohereRerank(model="rerank-multilingual-v3.0", top_n=2)

#         compression_retriever = ContextualCompressionRetriever(
#             base_compressor=compressor,
#             base_retriever=query_system.retriever
#         )
#         return compression_retriever
    
#     except (ImportError, ValueError) as e:
#         print(f"Lỗi khi cấu hình Cohere Rerank: {e}")
#         print(" => Bỏ qua.Sử dụng retriever cơ bản.")
#         return None

In [5]:
class QuerySystem:
    def __init__(self, config):
        self.config = config
        print("Đang khởi tạo")            
        self.embedding_model = SentenceTransformer(self.config['embedding_model'])        
        self.reranker = self._load_reranker()

        self.index = faiss.read_index(self.config['index_path'])                
        with open(self.config['docs_path'], "rb") as f:
            self.docs = pickle.load(f)            

    def _load_reranker(self):
        cohere_api_key = os.environ.get("COHERE_API_KEY")
        if COHERE_AVAILABLE and cohere_api_key:
            print("Có thể áp dụng Cohere Reranker.")
            return CohereRerank(
                cohere_api_key=cohere_api_key,
                model="rerank-multilingual-v3.0",
                top_n=self.config["reranker_top_n"]
            )
        else:
            print(f"Không tìm thấy COHERE_API_KEY. Sử dụng CrossEncoder: {self.config['reranker_model']}")
            return CrossEncoder(self.config['reranker_model'])

    def query(self, text: str) -> list[Document]:        
        query_vector = self.embedding_model.encode(text)
        query_vector_2d = np.array([query_vector], dtype='float32')
                
        distances, indices = self.index.search(query_vector_2d, k=self.config["retriever_k"])        
        retrieved_docs_data = [self.docs[i] for i in indices[0]]
        
        retrieved_docs_lc = [
            Document(page_content=doc['content'], metadata=doc['metadata']) 
            for doc in retrieved_docs_data
        ]
        
        print(f"\nCÂU HỎI: '{text}'")
        print(f"Đã truy vấn được {len(retrieved_docs_lc)} tài liệu ban đầu.")

        # ---RERANKING---
        if not self.reranker:
            print("Không có reranker, trả về kết quả truy vấn cơ bản.")
            return retrieved_docs_lc

        print("Đang thực hiện Reranking...")
        if isinstance(self.reranker, CohereRerank):
            reranked_docs = self.reranker.compress_documents(
                documents=retrieved_docs_lc,
                query=text
            )
        else: 
            reranker_input = [[text, doc.page_content] for doc in retrieved_docs_lc]
            scores = self.reranker.predict(reranker_input)
            
            doc_score_pairs = list(zip(retrieved_docs_lc, scores))
            doc_score_pairs.sort(key=lambda x: x[1], reverse=True)
            
            reranked_docs = [doc for doc, score in doc_score_pairs[:self.config["reranker_top_n"]]]

        return reranked_docs

In [8]:
CONFIG = {
    "index_path": "../index.faiss",  
    "docs_path": "../docs.pkl",      
    "embedding_model": "sentence-transformers/all-MiniLM-L6-v2",
    "reranker_model": "BAAI/bge-reranker-base",
    "retriever_k": 5,
    "reranker_top_n": 3
}

if not (os.path.exists(CONFIG['index_path']) and os.path.exists(CONFIG['docs_path'])):
    print(f"LỖI: Không tìm thấy file tại '{CONFIG['index_path']}' hoặc '{CONFIG['docs_path']}'.")
else:    
    query_system = QuerySystem(CONFIG)        
    user_query = "sinh viên được phép nghỉ học không?"    
    final_results = query_system.query(user_query)
        
    print(f"\nKết quả cuối cùng sau khi Reranking (top {len(final_results)}):")
    for i, doc in enumerate(final_results):
        print(f"  ------------ Document {i+1} -------------")
        print(f"  Nội dung: {doc.page_content}")        

Đang khởi tạo
Có thể áp dụng Cohere Reranker.

CÂU HỎI: 'sinh viên được phép nghỉ học không?'
Đã truy vấn được 5 tài liệu ban đầu.
Đang thực hiện Reranking...

Kết quả cuối cùng sau khi Reranking (top 3):
  ------------ Document 1 -------------
  Nội dung: khóa học và ổn định từ đầu đến cuối mỗi khóa. Phụ trách lớp khóa học là giáo viên 
chủ nhiệm. Đại diện lớp khóa học là Ban cán sự lớp. 
b) Lớp khóa học được gọi tên theo ngành học và năm nhập học của sinh viên, có mã 
hiệu theo quy định của Đại học Quốc gia Hà Nội. 
c) Sinh viên được phép nghỉ học tạm thời, khi trở lại học tiếp, được bố trí vào lớp 
khóa học phù hợp với khối lượng kiến thức đã tích lũy nhưng giữ nguyên mã sinh viên 
đã được cấp. Trường hợp sinh viên được bố trí vào lớp khóa học có những học phần 
trong chương trình đào tạo khác biệt so với chương trình đào tạo mà sinh viên đã 
theo học trước khi nghỉ học tạm thời, thủ trưởng đơn vị đào tạo quyết định cho sinh 
viên được bảo lưu hoặc phải học bổ sung những học phần tư